In [ ]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Display settings
pd.set_option('display.max_columns', 50)
sns.set_style('whitegrid')

## Random Forest - Premier League Standings Prediction

Predict final league positions (1-20) using Random Forest ensemble regressor.

Part of algorithm comparison series - results will be compared with K-Means, DBSCAN, SVM, Decision Tree, and XGBoost.

## Random Forest - Premier League Standings Prediction

Predict final league positions (1-20) using Random Forest ensemble regressor.

Part of algorithm comparison series - results will be compared with K-Means, DBSCAN, SVM, Decision Tree, and XGBoost.

In [ ]:
# Load the aggregated team-season dataset
from pathlib import Path
import pandas as pd
import numpy as np

candidate_paths = [
    Path('data/processed/team_season_aggregated.csv'),
    Path('../data/processed/team_season_aggregated.csv'),
    Path('../../data/processed/team_season_aggregated.csv'),
]

agg_path = next((p for p in candidate_paths if p.exists()), None)
if agg_path is None:
    raise FileNotFoundError(
        'team_season_aggregated.csv not found! '
        'Please run data_preprocessing.ipynb first to create this file.'
    )

print(f'✅ Loading aggregated dataset: {agg_path}')
df_agg = pd.read_csv(agg_path)

print(f'\nDataset shape: {df_agg.shape}')
print(f'Seasons: {sorted(df_agg["Season"].unique())}')
print(f'Teams per season: ~{len(df_agg) // df_agg["Season"].nunique()}')

print('\nSample data:')
print(df_agg.head())

print('\nTarget variable (Final_Position) distribution:')
print(df_agg['Final_Position'].value_counts().sort_index())

✅ Loading aggregated dataset: ..\..\data\processed\team_season_aggregated.csv

Dataset shape: (500, 35)
Seasons: ['2000-01', '2001-02', '2002-03', '2003-04', '2004-05', '2005-06', '2006-07', '2007-08', '2008-09', '2009-10', '2010-11', '2011-12', '2012-13', '2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22', '2022-23', '2023-24', '2024-25']
Teams per season: ~20

Sample data:
    Season  Season_encoded        Team  Matches_Played  Home_Matches  \
0  2000-01               0  Man United              38            19   
1  2000-01               0     Arsenal              38            19   
2  2000-01               0   Liverpool              38            19   
3  2000-01               0       Leeds              38            19   
4  2000-01               0     Ipswich              38            19   

   Away_Matches  Wins  Draws  Losses  Home_Wins  Away_Wins  Points  \
0            19    24      8       6         15          9      80   
1 